

# Project-5

# Formal Problem Statement
Variable names are self explanatory and there is no formal data dictionary provided by the client .
Your task here is to build a predictive model for predicting whether a property should be marked
as junk on the basis of listing details and preliminary assessment details. You need to build your
model on the train dataset. Test dataset does not have a response column; you need to
predict those values and submit it in a csv format.

# target = Junk

In [59]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import xgboost as xgb

from sklearn.model_selection import RandomizedSearchCV,train_test_split

from xgboost.sklearn import XGBClassifier 

In [60]:
datafile_train=r"C:\Users\sindhuja\Desktop\python\project\project-5\Property_train.csv"
datafile_test=r"C:\Users\sindhuja\Desktop\python\project\project-5\Property_test_share.csv"
bd_train=pd.read_csv(datafile_train)
bd_test=pd.read_csv(datafile_test)

In [61]:
bd_train.head()

,Junk,InteriorsStyle,PriceIndex8,ListDate,Material,PriceIndex9,Agency,AreaIncomeType,EnvRating,PriceIndex7,...,Region,PriceIndex5,SubModel,Facade,State,NormalisedPopulation,BuildYear,RegionType,PropertyAge,PriceIndex2
0,0,6LLJ,14674.0,9/7/2010,UT7W,8270,CAT3,B,missing,13143.0,...,A,9022.0,6E9G,GREEN,MD,42077,2008,A,2,10692.0
1,1,XZ9F,4172.0,1/7/2009,PRN0,3890,CAT2,B,missing,3461.0,...,A,2742.0,FIVQ,SILVER,OK,77258,2002,A,7,3615.0
2,0,QWEV,8210.0,6/23/2010,A9RA,5900,CAT1,B,missing,6922.0,...,A,3768.0,4K0H,SILVER,SC,94514,2004,A,6,5805.0
3,0,MK2K,7309.0,3/19/2009,RRJQ,7460,OTHER,A,missing,6300.0,...,A,5370.0,ZLXY,WHITE,NC,82302,2002,C,7,6444.0
4,0,GK4G,9182.0,3/3/2010,HE5A,6400,OTHER,B,missing,8330.0,...,A,4803.0,FIVQ,SILVER,SC,49176,2007,B,3,6286.0


In [62]:
bd_train.shape

(62035, 31)

In [63]:
bd_test.shape

(10948, 30)

In [64]:
bd_test['Junk']=np.nan

In [65]:
bd_test.head()

,InteriorsStyle,PriceIndex8,ListDate,Material,PriceIndex9,Agency,AreaIncomeType,EnvRating,PriceIndex7,ExpeditedListing,...,PriceIndex5,SubModel,Facade,State,NormalisedPopulation,BuildYear,RegionType,PropertyAge,PriceIndex2,Junk
0,6LLJ,10645.0,9/22/2010,ZCI3,6015,CAT3,A,missing,9568.0,0,...,5911.0,UA42,BLUE,NC,67549,2006,A,4,8311.0,NaN
1,QD0H,9891.0,3/12/2010,RRJQ,7625,OTHER,A,missing,8888.0,0,...,5370.0,BUNS,BLUE,FL,56570,2004,B,6,7123.0,NaN
2,I7GR,8426.0,10/12/2009,ZCI3,6900,CAT2,B,missing,7911.0,0,...,6862.0,8YTZ,BLACK,OK,51840,2008,A,1,7994.0,NaN
3,5KS5,9269.0,5/20/2010,PRN0,6475,CAT1,B,missing,7773.0,0,...,4872.0,FIVQ,GREEN,CO,83300,2005,A,5,5259.0,NaN
4,918O,5366.0,1/13/2009,C8A4,3960,CAT1,A,missing,4134.0,0,...,3365.0,KT8F,BLACK,NC,95035,2003,A,6,4362.0,NaN


In [66]:
bd_train['data']='train'
bd_test['data']='test'

In [67]:
bd_test=bd_test[bd_train.columns]  # the columns in the two data frames should be in the same order to enable concatenation

In [68]:
bd_all=pd.concat([bd_train,bd_test],axis=0)

In [69]:
bd_all.head(2)

,Junk,InteriorsStyle,PriceIndex8,ListDate,Material,PriceIndex9,Agency,AreaIncomeType,EnvRating,PriceIndex7,...,PriceIndex5,SubModel,Facade,State,NormalisedPopulation,BuildYear,RegionType,PropertyAge,PriceIndex2,data
0,0.0,6LLJ,14674.0,9/7/2010,UT7W,8270,CAT3,B,missing,13143.0,...,9022.0,6E9G,GREEN,MD,42077,2008,A,2,10692.0,train
1,1.0,XZ9F,4172.0,1/7/2009,PRN0,3890,CAT2,B,missing,3461.0,...,2742.0,FIVQ,SILVER,OK,77258,2002,A,7,3615.0,train


In [70]:
bd_all.shape

(72983, 32)

In [71]:
bd_all.dtypes

Junk                     float64
InteriorsStyle            object
PriceIndex8               object
ListDate                  object
Material                  object
PriceIndex9                int64
Agency                    object
AreaIncomeType            object
EnvRating                 object
PriceIndex7               object
ExpeditedListing           int64
PriceIndex4               object
PriceIndex1               object
PriceIndex6               object
PRIMEUNIT                 object
Channel                   object
Zip                        int64
InsurancePremiumIndex      int64
PlotType                  object
Architecture              object
PriceIndex3               object
Region                    object
PriceIndex5               object
SubModel                  object
Facade                    object
State                     object
NormalisedPopulation       int64
BuildYear                  int64
RegionType                object
PropertyAge                int64
PriceIndex

In [72]:
list(zip(bd_all.columns,bd_all.dtypes,bd_all.nunique()))

[('Junk', dtype('float64'), 2),
 ('InteriorsStyle', dtype('O'), 1063),
 ('PriceIndex8', dtype('O'), 13193),
 ('ListDate', dtype('O'), 517),
 ('Material', dtype('O'), 135),
 ('PriceIndex9', dtype('int64'), 2061),
 ('Agency', dtype('O'), 5),
 ('AreaIncomeType', dtype('O'), 4),
 ('EnvRating', dtype('O'), 3),
 ('PriceIndex7', dtype('O'), 12494),
 ('ExpeditedListing', dtype('int64'), 2),
 ('PriceIndex4', dtype('O'), 13457),
 ('PriceIndex1', dtype('O'), 10343),
 ('PriceIndex6', dtype('O'), 11266),
 ('PRIMEUNIT', dtype('O'), 3),
 ('Channel', dtype('O'), 3),
 ('Zip', dtype('int64'), 153),
 ('InsurancePremiumIndex', dtype('int64'), 281),
 ('PlotType', dtype('O'), 13),
 ('Architecture', dtype('O'), 33),
 ('PriceIndex3', dtype('O'), 12726),
 ('Region', dtype('O'), 3),
 ('PriceIndex5', dtype('O'), 10316),
 ('SubModel', dtype('O'), 864),
 ('Facade', dtype('O'), 17),
 ('State', dtype('O'), 37),
 ('NormalisedPopulation', dtype('int64'), 39947),
 ('BuildYear', dtype('int64'), 10),
 ('RegionType', dtyp

### create dummies with freq cut off


cols = ['V1','V4','V5','V6','V44']

    for col in cols:
    k = cd_all[col].value_counts()
    k = k[k>50]
    indexes = k.index[:-1]
     for i in indexes:
            cd_all[col+'_'+str(i)] = (cd_all[col]==i).astype(int)

### 1.Junk

In [73]:
bd_all.Junk.value_counts()

0.0    54433
1.0     7602
Name: Junk, dtype: int64

In [74]:
# here it is the target

### 2.InteriorsStyle

In [75]:
bd_all.shape

(72983, 32)

In [76]:
r=bd_all.InteriorsStyle.value_counts()
r

K3BB    2329
RCQE    1990
WEGU    1425
2XWO    1375
V3Y1    1289
        ... 
57J5       1
YOVG       1
JIOH       1
XY4G       1
ALFW       1
Name: InteriorsStyle, Length: 1063, dtype: int64

In [77]:
(r>1000).astype(int).sum()

12

In [78]:
# create dummies with freq cut off as 1000

In [79]:
r[r>1000].index

Index(['K3BB', 'RCQE', 'WEGU', '2XWO', 'V3Y1', 'YBQJ', 'AZMX', 'DUFA', '7R8J',
       'H1CI', 'ARWN', 'T67A'],
      dtype='object')

In [80]:

cols = ['InteriorsStyle']
for col in cols:
    k = bd_all[col].value_counts()
    k = k[k>1000]
    indexes = k.index[:-1]
    print(indexes)
    for i in indexes:
        print(i)
        bd_all[col+'_'+str(i)] = (bd_all[col]==i).astype(int)

Index(['K3BB', 'RCQE', 'WEGU', '2XWO', 'V3Y1', 'YBQJ', 'AZMX', 'DUFA', '7R8J',
       'H1CI', 'ARWN'],
      dtype='object')
K3BB
RCQE
WEGU
2XWO
V3Y1
YBQJ
AZMX
DUFA
7R8J
H1CI
ARWN


In [81]:
bd_all.head(2)

,Junk,InteriorsStyle,PriceIndex8,ListDate,Material,PriceIndex9,Agency,AreaIncomeType,EnvRating,PriceIndex7,...,InteriorsStyle_RCQE,InteriorsStyle_WEGU,InteriorsStyle_2XWO,InteriorsStyle_V3Y1,InteriorsStyle_YBQJ,InteriorsStyle_AZMX,InteriorsStyle_DUFA,InteriorsStyle_7R8J,InteriorsStyle_H1CI,InteriorsStyle_ARWN
0,0.0,6LLJ,14674.0,9/7/2010,UT7W,8270,CAT3,B,missing,13143.0,...,0,0,0,0,0,0,0,0,0,0
1,1.0,XZ9F,4172.0,1/7/2009,PRN0,3890,CAT2,B,missing,3461.0,...,0,0,0,0,0,0,0,0,0,0


### 3.PriceIndex8

In [82]:
(bd_all.PriceIndex8.value_counts()>100).sum()

7

In [83]:
bd_all.PriceIndex8.head()

0    14674.0
1     4172.0
2     8210.0
3     7309.0
4     9182.0
Name: PriceIndex8, dtype: object

In [84]:
# convert to numeric

In [85]:
bd_all.columns

Index(['Junk', 'InteriorsStyle', 'PriceIndex8', 'ListDate', 'Material',
       'PriceIndex9', 'Agency', 'AreaIncomeType', 'EnvRating', 'PriceIndex7',
       'ExpeditedListing', 'PriceIndex4', 'PriceIndex1', 'PriceIndex6',
       'PRIMEUNIT', 'Channel', 'Zip', 'InsurancePremiumIndex', 'PlotType',
       'Architecture', 'PriceIndex3', 'Region', 'PriceIndex5', 'SubModel',
       'Facade', 'State', 'NormalisedPopulation', 'BuildYear', 'RegionType',
       'PropertyAge', 'PriceIndex2', 'data', 'InteriorsStyle_K3BB',
       'InteriorsStyle_RCQE', 'InteriorsStyle_WEGU', 'InteriorsStyle_2XWO',
       'InteriorsStyle_V3Y1', 'InteriorsStyle_YBQJ', 'InteriorsStyle_AZMX',
       'InteriorsStyle_DUFA', 'InteriorsStyle_7R8J', 'InteriorsStyle_H1CI',
       'InteriorsStyle_ARWN'],
      dtype='object')

In [86]:
bd_all.PriceIndex8=pd.to_numeric(bd_all.PriceIndex8,errors='coerce')

In [87]:
bd_all.PriceIndex8.dtype

dtype('float64')

### 4.ListDate

In [88]:
bd_all.ListDate.value_counts()

11/23/2010    384
2/25/2009     379
12/8/2010     372
10/13/2010    359
8/26/2009     359
             ... 
2/20/2010       2
5/25/2009       2
1/10/2010       1
12/24/2010      1
12/4/2010       1
Name: ListDate, Length: 517, dtype: int64

In [89]:
(bd_all.ListDate.value_counts()>300).sum()

20

In [90]:
#convert this to month

In [91]:
m=bd_all.ListDate


m['date']=pd.to_datetime(m)


m['month']=m['date'].dt.month



In [92]:
bd_all['ListDate_converted']=m['month']

In [93]:
bd_all.head(2)

,Junk,InteriorsStyle,PriceIndex8,ListDate,Material,PriceIndex9,Agency,AreaIncomeType,EnvRating,PriceIndex7,...,InteriorsStyle_WEGU,InteriorsStyle_2XWO,InteriorsStyle_V3Y1,InteriorsStyle_YBQJ,InteriorsStyle_AZMX,InteriorsStyle_DUFA,InteriorsStyle_7R8J,InteriorsStyle_H1CI,InteriorsStyle_ARWN,ListDate_converted
0,0.0,6LLJ,14674.0,9/7/2010,UT7W,8270,CAT3,B,missing,13143.0,...,0,0,0,0,0,0,0,0,0,9
1,1.0,XZ9F,4172.0,1/7/2009,PRN0,3890,CAT2,B,missing,3461.0,...,0,0,0,0,0,0,0,0,0,1


In [94]:
bd_all['ListDate_converted'].value_counts()

10    7178
2     6839
11    6675
9     6511
3     6257
8     6139
4     5935
7     5876
5     5760
6     5675
12    5368
1     4770
Name: ListDate_converted, dtype: int64

In [95]:
bd_all=pd.get_dummies(bd_all,columns=['ListDate_converted'],drop_first=True)

In [96]:
bd_all.columns

Index(['Junk', 'InteriorsStyle', 'PriceIndex8', 'ListDate', 'Material',
       'PriceIndex9', 'Agency', 'AreaIncomeType', 'EnvRating', 'PriceIndex7',
       'ExpeditedListing', 'PriceIndex4', 'PriceIndex1', 'PriceIndex6',
       'PRIMEUNIT', 'Channel', 'Zip', 'InsurancePremiumIndex', 'PlotType',
       'Architecture', 'PriceIndex3', 'Region', 'PriceIndex5', 'SubModel',
       'Facade', 'State', 'NormalisedPopulation', 'BuildYear', 'RegionType',
       'PropertyAge', 'PriceIndex2', 'data', 'InteriorsStyle_K3BB',
       'InteriorsStyle_RCQE', 'InteriorsStyle_WEGU', 'InteriorsStyle_2XWO',
       'InteriorsStyle_V3Y1', 'InteriorsStyle_YBQJ', 'InteriorsStyle_AZMX',
       'InteriorsStyle_DUFA', 'InteriorsStyle_7R8J', 'InteriorsStyle_H1CI',
       'InteriorsStyle_ARWN', 'ListDate_converted_2', 'ListDate_converted_3',
       'ListDate_converted_4', 'ListDate_converted_5', 'ListDate_converted_6',
       'ListDate_converted_7', 'ListDate_converted_8', 'ListDate_converted_9',
       'ListDate_co

### 5.Material

In [97]:
bd_all['Material'].value_counts()

PRN0    13950
C8A4    10174
ZCI3     9348
NYFD     3825
X7IX     3540
        ...  
MS1W        1
0EFS        1
HTTM        1
M81Y        1
VBPN        1
Name: Material, Length: 135, dtype: int64

In [98]:
(bd_all['Material'].value_counts()>3000).sum()

5

In [99]:
bd_all.shape

(72983, 54)

In [100]:
# create dummies with feq cutoff of 3000 which is almost 5% of population

In [101]:
cols = ['Material']
for col in cols:
    k = bd_all[col].value_counts()
    k = k[k>3000]
    indexes = k.index[:-1]
    print(indexes)
    for i in indexes:
        print(i)
        bd_all[col+'_'+str(i)] = (bd_all[col]==i).astype(int)

Index(['PRN0', 'C8A4', 'ZCI3', 'NYFD'], dtype='object')
PRN0
C8A4
ZCI3
NYFD


In [102]:
bd_all.columns

Index(['Junk', 'InteriorsStyle', 'PriceIndex8', 'ListDate', 'Material',
       'PriceIndex9', 'Agency', 'AreaIncomeType', 'EnvRating', 'PriceIndex7',
       'ExpeditedListing', 'PriceIndex4', 'PriceIndex1', 'PriceIndex6',
       'PRIMEUNIT', 'Channel', 'Zip', 'InsurancePremiumIndex', 'PlotType',
       'Architecture', 'PriceIndex3', 'Region', 'PriceIndex5', 'SubModel',
       'Facade', 'State', 'NormalisedPopulation', 'BuildYear', 'RegionType',
       'PropertyAge', 'PriceIndex2', 'data', 'InteriorsStyle_K3BB',
       'InteriorsStyle_RCQE', 'InteriorsStyle_WEGU', 'InteriorsStyle_2XWO',
       'InteriorsStyle_V3Y1', 'InteriorsStyle_YBQJ', 'InteriorsStyle_AZMX',
       'InteriorsStyle_DUFA', 'InteriorsStyle_7R8J', 'InteriorsStyle_H1CI',
       'InteriorsStyle_ARWN', 'ListDate_converted_2', 'ListDate_converted_3',
       'ListDate_converted_4', 'ListDate_converted_5', 'ListDate_converted_6',
       'ListDate_converted_7', 'ListDate_converted_8', 'ListDate_converted_9',
       'ListDate_co

In [103]:
bd_all.head(3)

,Junk,InteriorsStyle,PriceIndex8,ListDate,Material,PriceIndex9,Agency,AreaIncomeType,EnvRating,PriceIndex7,...,ListDate_converted_7,ListDate_converted_8,ListDate_converted_9,ListDate_converted_10,ListDate_converted_11,ListDate_converted_12,Material_PRN0,Material_C8A4,Material_ZCI3,Material_NYFD
0,0.0,6LLJ,14674.0,9/7/2010,UT7W,8270,CAT3,B,missing,13143.0,...,0,0,1,0,0,0,0,0,0,0
1,1.0,XZ9F,4172.0,1/7/2009,PRN0,3890,CAT2,B,missing,3461.0,...,0,0,0,0,0,0,1,0,0,0
2,0.0,QWEV,8210.0,6/23/2010,A9RA,5900,CAT1,B,missing,6922.0,...,0,0,0,0,0,0,0,0,0,0


### 6.PriceIndex9

In [104]:
bd_all.PriceIndex9.dtype

dtype('int64')

In [105]:
#no changes required

### 7.Agency

In [106]:
bd_all.Agency.value_counts()

CAT1       25314
CAT2       23399
CAT3       12315
OTHER      11950
missing        5
Name: Agency, dtype: int64

In [109]:
#convert to dummies

In [107]:

cols = ['Agency']
for col in cols:
    k = bd_all[col].value_counts()
    k = k[k>100]
    indexes = k.index[:-1]
    print(indexes)
    for i in indexes:
        print(i)
        bd_all[col+'_'+str(i)] = (bd_all[col]==i).astype(int)

Index(['CAT1', 'CAT2', 'CAT3'], dtype='object')
CAT1
CAT2
CAT3


In [108]:
bd_all.columns

Index(['Junk', 'InteriorsStyle', 'PriceIndex8', 'ListDate', 'Material',
       'PriceIndex9', 'Agency', 'AreaIncomeType', 'EnvRating', 'PriceIndex7',
       'ExpeditedListing', 'PriceIndex4', 'PriceIndex1', 'PriceIndex6',
       'PRIMEUNIT', 'Channel', 'Zip', 'InsurancePremiumIndex', 'PlotType',
       'Architecture', 'PriceIndex3', 'Region', 'PriceIndex5', 'SubModel',
       'Facade', 'State', 'NormalisedPopulation', 'BuildYear', 'RegionType',
       'PropertyAge', 'PriceIndex2', 'data', 'InteriorsStyle_K3BB',
       'InteriorsStyle_RCQE', 'InteriorsStyle_WEGU', 'InteriorsStyle_2XWO',
       'InteriorsStyle_V3Y1', 'InteriorsStyle_YBQJ', 'InteriorsStyle_AZMX',
       'InteriorsStyle_DUFA', 'InteriorsStyle_7R8J', 'InteriorsStyle_H1CI',
       'InteriorsStyle_ARWN', 'ListDate_converted_2', 'ListDate_converted_3',
       'ListDate_converted_4', 'ListDate_converted_5', 'ListDate_converted_6',
       'ListDate_converted_7', 'ListDate_converted_8', 'ListDate_converted_9',
       'ListDate_co

### 8.AreaIncomeType

In [111]:
bd_all.AreaIncomeType.value_counts()

A          36050
B          33004
missing     3174
C            755
Name: AreaIncomeType, dtype: int64

In [ ]:
#create dummies with the cut off 3000

In [112]:

cols = ['AreaIncomeType']
for col in cols:
    k = bd_all[col].value_counts()
    k = k[k>3000]
    indexes = k.index[:-1]
    print(indexes)
    for i in indexes:
        print(i)
        bd_all[col+'_'+str(i)] = (bd_all[col]==i).astype(int)

Index(['A', 'B'], dtype='object')
A
B


In [113]:
bd_all.columns

Index(['Junk', 'InteriorsStyle', 'PriceIndex8', 'ListDate', 'Material',
       'PriceIndex9', 'Agency', 'AreaIncomeType', 'EnvRating', 'PriceIndex7',
       'ExpeditedListing', 'PriceIndex4', 'PriceIndex1', 'PriceIndex6',
       'PRIMEUNIT', 'Channel', 'Zip', 'InsurancePremiumIndex', 'PlotType',
       'Architecture', 'PriceIndex3', 'Region', 'PriceIndex5', 'SubModel',
       'Facade', 'State', 'NormalisedPopulation', 'BuildYear', 'RegionType',
       'PropertyAge', 'PriceIndex2', 'data', 'InteriorsStyle_K3BB',
       'InteriorsStyle_RCQE', 'InteriorsStyle_WEGU', 'InteriorsStyle_2XWO',
       'InteriorsStyle_V3Y1', 'InteriorsStyle_YBQJ', 'InteriorsStyle_AZMX',
       'InteriorsStyle_DUFA', 'InteriorsStyle_7R8J', 'InteriorsStyle_H1CI',
       'InteriorsStyle_ARWN', 'ListDate_converted_2', 'ListDate_converted_3',
       'ListDate_converted_4', 'ListDate_converted_5', 'ListDate_converted_6',
       'ListDate_converted_7', 'ListDate_converted_8', 'ListDate_converted_9',
       'ListDate_co

### 9.EnvRating

In [115]:
bd_all.EnvRating.value_counts()

missing    69564
GREEN       3340
RED           79
Name: EnvRating, dtype: int64

In [116]:
#create dummies with the cutoff of 3000

In [117]:

cols = ['EnvRating']
for col in cols:
    k = bd_all[col].value_counts()
    k = k[k>3000]
    indexes = k.index[:-1]
    print(indexes)
    for i in indexes:
        print(i)
        bd_all[col+'_'+str(i)] = (bd_all[col]==i).astype(int)

Index(['missing'], dtype='object')
missing


In [119]:
bd_all.columns

Index(['Junk', 'InteriorsStyle', 'PriceIndex8', 'ListDate', 'Material',
       'PriceIndex9', 'Agency', 'AreaIncomeType', 'EnvRating', 'PriceIndex7',
       'ExpeditedListing', 'PriceIndex4', 'PriceIndex1', 'PriceIndex6',
       'PRIMEUNIT', 'Channel', 'Zip', 'InsurancePremiumIndex', 'PlotType',
       'Architecture', 'PriceIndex3', 'Region', 'PriceIndex5', 'SubModel',
       'Facade', 'State', 'NormalisedPopulation', 'BuildYear', 'RegionType',
       'PropertyAge', 'PriceIndex2', 'data', 'InteriorsStyle_K3BB',
       'InteriorsStyle_RCQE', 'InteriorsStyle_WEGU', 'InteriorsStyle_2XWO',
       'InteriorsStyle_V3Y1', 'InteriorsStyle_YBQJ', 'InteriorsStyle_AZMX',
       'InteriorsStyle_DUFA', 'InteriorsStyle_7R8J', 'InteriorsStyle_H1CI',
       'InteriorsStyle_ARWN', 'ListDate_converted_2', 'ListDate_converted_3',
       'ListDate_converted_4', 'ListDate_converted_5', 'ListDate_converted_6',
       'ListDate_converted_7', 'ListDate_converted_8', 'ListDate_converted_9',
       'ListDate_co

### 10.PriceIndex7

In [120]:
bd_all.PriceIndex7

0        13143.0
1         3461.0
2         6922.0
3         6300.0
4         8330.0
          ...   
10943    12860.0
10944     6845.0
10945    14663.0
10946    10292.0
10947     3524.0
Name: PriceIndex7, Length: 72983, dtype: object

In [121]:
#convert to numeric

In [122]:
bd_all.PriceIndex7=pd.to_numeric(bd_all.PriceIndex7,errors='coerce')

In [124]:
bd_all.PriceIndex7.dtype

dtype('float64')

### 11.ExpeditedListing

In [125]:
bd_all.ExpeditedListing

0        0
1        0
2        0
3        0
4        0
        ..
10943    0
10944    0
10945    0
10946    0
10947    0
Name: ExpeditedListing, Length: 72983, dtype: int64

In [126]:
bd_all.ExpeditedListing.value_counts()

0    71138
1     1845
Name: ExpeditedListing, dtype: int64

In [127]:
#no changes required

### 12.PriceIndex4

In [128]:
bd_all.PriceIndex4

0        14224.0
1         4404.0
2         8952.0
3         7460.0
4         9846.0
          ...   
10943    10415.0
10944     8521.0
10945    13555.0
10946    11733.0
10947     4223.0
Name: PriceIndex4, Length: 72983, dtype: object

In [129]:
#convert to numeric

In [130]:
bd_all.PriceIndex4=pd.to_numeric(bd_all.PriceIndex4,errors='coerce')

In [131]:
bd_all.PriceIndex4.dtype

dtype('float64')

In [133]:
bd_all.columns

Index(['Junk', 'InteriorsStyle', 'PriceIndex8', 'ListDate', 'Material',
       'PriceIndex9', 'Agency', 'AreaIncomeType', 'EnvRating', 'PriceIndex7',
       'ExpeditedListing', 'PriceIndex4', 'PriceIndex1', 'PriceIndex6',
       'PRIMEUNIT', 'Channel', 'Zip', 'InsurancePremiumIndex', 'PlotType',
       'Architecture', 'PriceIndex3', 'Region', 'PriceIndex5', 'SubModel',
       'Facade', 'State', 'NormalisedPopulation', 'BuildYear', 'RegionType',
       'PropertyAge', 'PriceIndex2', 'data', 'InteriorsStyle_K3BB',
       'InteriorsStyle_RCQE', 'InteriorsStyle_WEGU', 'InteriorsStyle_2XWO',
       'InteriorsStyle_V3Y1', 'InteriorsStyle_YBQJ', 'InteriorsStyle_AZMX',
       'InteriorsStyle_DUFA', 'InteriorsStyle_7R8J', 'InteriorsStyle_H1CI',
       'InteriorsStyle_ARWN', 'ListDate_converted_2', 'ListDate_converted_3',
       'ListDate_converted_4', 'ListDate_converted_5', 'ListDate_converted_6',
       'ListDate_converted_7', 'ListDate_converted_8', 'ListDate_converted_9',
       'ListDate_co

### 13.PriceIndex1

In [134]:
bd_all.PriceIndex1

0         9217.0
1         2958.0
2         4568.0
3         5361.0
4         5428.0
          ...   
10943     8497.0
10944     3739.0
10945    10199.0
10946     6735.0
10947     2547.0
Name: PriceIndex1, Length: 72983, dtype: object

In [135]:
#convert to numeric

In [136]:
bd_all.PriceIndex1=pd.to_numeric(bd_all.PriceIndex1,errors='coerce')

In [137]:
bd_all.PriceIndex1.dtype

dtype('float64')

### 14.PriceIndex6

In [138]:
bd_all.PriceIndex6

0        10387.0
1         3400.0
2         4862.0
3         6305.0
4         5718.0
          ...   
10943     9703.0
10944     4716.0
10945    11939.0
10946     8075.0
10947     3849.0
Name: PriceIndex6, Length: 72983, dtype: object

In [139]:
#conert to numeric

In [140]:
bd_all.PriceIndex6=pd.to_numeric(bd_all.PriceIndex6,errors='coerce')

In [141]:
bd_all.PriceIndex6.dtype

dtype('float64')

### 15.PRIMEUNIT

In [143]:
bd_all.PRIMEUNIT.value_counts()

missing    69564
NO          3357
YES           62
Name: PRIMEUNIT, dtype: int64

In [ ]:
#create dummies with the cut off of 3000

In [144]:

cols = ['PRIMEUNIT']
for col in cols:
    k = bd_all[col].value_counts()
    k = k[k>3000]
    indexes = k.index[:-1]
    print(indexes)
    for i in indexes:
        print(i)
        bd_all[col+'_'+str(i)] = (bd_all[col]==i).astype(int)

Index(['missing'], dtype='object')
missing


In [145]:
bd_all.columns

Index(['Junk', 'InteriorsStyle', 'PriceIndex8', 'ListDate', 'Material',
       'PriceIndex9', 'Agency', 'AreaIncomeType', 'EnvRating', 'PriceIndex7',
       'ExpeditedListing', 'PriceIndex4', 'PriceIndex1', 'PriceIndex6',
       'PRIMEUNIT', 'Channel', 'Zip', 'InsurancePremiumIndex', 'PlotType',
       'Architecture', 'PriceIndex3', 'Region', 'PriceIndex5', 'SubModel',
       'Facade', 'State', 'NormalisedPopulation', 'BuildYear', 'RegionType',
       'PropertyAge', 'PriceIndex2', 'data', 'InteriorsStyle_K3BB',
       'InteriorsStyle_RCQE', 'InteriorsStyle_WEGU', 'InteriorsStyle_2XWO',
       'InteriorsStyle_V3Y1', 'InteriorsStyle_YBQJ', 'InteriorsStyle_AZMX',
       'InteriorsStyle_DUFA', 'InteriorsStyle_7R8J', 'InteriorsStyle_H1CI',
       'InteriorsStyle_ARWN', 'ListDate_converted_2', 'ListDate_converted_3',
       'ListDate_converted_4', 'ListDate_converted_5', 'ListDate_converted_6',
       'ListDate_converted_7', 'ListDate_converted_8', 'ListDate_converted_9',
       'ListDate_co

### 16.Channel

In [147]:
bd_all.Channel.value_counts()

Direct    41043
Other     17501
Agent     14439
Name: Channel, dtype: int64

In [148]:
#create dummies

In [149]:
cols = ['Channel']
for col in cols:
    k = bd_all[col].value_counts()
    k = k[k>1000]
    indexes = k.index[:-1]
    print(indexes)
    for i in indexes:
        print(i)
        bd_all[col+'_'+str(i)] = (bd_all[col]==i).astype(int)

Index(['Direct', 'Other'], dtype='object')
Direct
Other


In [150]:
bd_all.columns

Index(['Junk', 'InteriorsStyle', 'PriceIndex8', 'ListDate', 'Material',
       'PriceIndex9', 'Agency', 'AreaIncomeType', 'EnvRating', 'PriceIndex7',
       'ExpeditedListing', 'PriceIndex4', 'PriceIndex1', 'PriceIndex6',
       'PRIMEUNIT', 'Channel', 'Zip', 'InsurancePremiumIndex', 'PlotType',
       'Architecture', 'PriceIndex3', 'Region', 'PriceIndex5', 'SubModel',
       'Facade', 'State', 'NormalisedPopulation', 'BuildYear', 'RegionType',
       'PropertyAge', 'PriceIndex2', 'data', 'InteriorsStyle_K3BB',
       'InteriorsStyle_RCQE', 'InteriorsStyle_WEGU', 'InteriorsStyle_2XWO',
       'InteriorsStyle_V3Y1', 'InteriorsStyle_YBQJ', 'InteriorsStyle_AZMX',
       'InteriorsStyle_DUFA', 'InteriorsStyle_7R8J', 'InteriorsStyle_H1CI',
       'InteriorsStyle_ARWN', 'ListDate_converted_2', 'ListDate_converted_3',
       'ListDate_converted_4', 'ListDate_converted_5', 'ListDate_converted_6',
       'ListDate_converted_7', 'ListDate_converted_8', 'ListDate_converted_9',
       'ListDate_co

### 16.Zip

In [152]:
bd_all.Zip.value_counts()

32824    3699
27542    3402
75236    2431
74135    2321
80022    2118
         ... 
85248       1
25071       1
85260       1
80112       1
76101       1
Name: Zip, Length: 153, dtype: int64

In [161]:
(bd_all.Zip.value_counts()>3000).sum()

2

In [157]:
#create dummies it is zip code  with cutoff 3000 since the counts should be atleast 5% of the population

In [159]:
cols = ['Zip']
for col in cols:
    k = bd_all[col].value_counts()
    k = k[k>3000]
    indexes = k.index[:-1]
    print(indexes)
    for i in indexes:
        print(i)
        bd_all[col+'_'+str(i)] = (bd_all[col]==i).astype(int)

Int64Index([32824], dtype='int64')
32824


In [160]:
bd_all.columns

Index(['Junk', 'InteriorsStyle', 'PriceIndex8', 'ListDate', 'Material',
       'PriceIndex9', 'Agency', 'AreaIncomeType', 'EnvRating', 'PriceIndex7',
       'ExpeditedListing', 'PriceIndex4', 'PriceIndex1', 'PriceIndex6',
       'PRIMEUNIT', 'Channel', 'Zip', 'InsurancePremiumIndex', 'PlotType',
       'Architecture', 'PriceIndex3', 'Region', 'PriceIndex5', 'SubModel',
       'Facade', 'State', 'NormalisedPopulation', 'BuildYear', 'RegionType',
       'PropertyAge', 'PriceIndex2', 'data', 'InteriorsStyle_K3BB',
       'InteriorsStyle_RCQE', 'InteriorsStyle_WEGU', 'InteriorsStyle_2XWO',
       'InteriorsStyle_V3Y1', 'InteriorsStyle_YBQJ', 'InteriorsStyle_AZMX',
       'InteriorsStyle_DUFA', 'InteriorsStyle_7R8J', 'InteriorsStyle_H1CI',
       'InteriorsStyle_ARWN', 'ListDate_converted_2', 'ListDate_converted_3',
       'ListDate_converted_4', 'ListDate_converted_5', 'ListDate_converted_6',
       'ListDate_converted_7', 'ListDate_converted_8', 'ListDate_converted_9',
       'ListDate_co

### 17.InsurancePremiumIndex

In [163]:
bd_all.InsurancePremiumIndex.value_counts()

920     2870
1974    2407
2152    2082
1389    2078
1215    2006
        ... 
1872       1
6519       1
1361       1
4241       1
1931       1
Name: InsurancePremiumIndex, Length: 281, dtype: int64

In [164]:
# no changes required

### 18.PlotType

In [166]:
bd_all.PlotType.value_counts()

WHBI    30785
W62B     8850
32T2     8090
DIJX     7205
WLQ6     5854
DSSM     3170
DJOP     2276
7RBQ     1915
I00H     1759
V5R0     1433
8FRL      864
LIGY      777
T559        5
Name: PlotType, dtype: int64

In [167]:
# create dummies with cutoff of 3000 

In [168]:
cols = ['PlotType']
for col in cols:
    k = bd_all[col].value_counts()
    k = k[k>3000]
    indexes = k.index[:-1]
    print(indexes)
    for i in indexes:
        print(i)
        bd_all[col+'_'+str(i)] = (bd_all[col]==i).astype(int)

Index(['WHBI', 'W62B', '32T2', 'DIJX', 'WLQ6'], dtype='object')
WHBI
W62B
32T2
DIJX
WLQ6


In [169]:
bd_all.columns

Index(['Junk', 'InteriorsStyle', 'PriceIndex8', 'ListDate', 'Material',
       'PriceIndex9', 'Agency', 'AreaIncomeType', 'EnvRating', 'PriceIndex7',
       'ExpeditedListing', 'PriceIndex4', 'PriceIndex1', 'PriceIndex6',
       'PRIMEUNIT', 'Channel', 'Zip', 'InsurancePremiumIndex', 'PlotType',
       'Architecture', 'PriceIndex3', 'Region', 'PriceIndex5', 'SubModel',
       'Facade', 'State', 'NormalisedPopulation', 'BuildYear', 'RegionType',
       'PropertyAge', 'PriceIndex2', 'data', 'InteriorsStyle_K3BB',
       'InteriorsStyle_RCQE', 'InteriorsStyle_WEGU', 'InteriorsStyle_2XWO',
       'InteriorsStyle_V3Y1', 'InteriorsStyle_YBQJ', 'InteriorsStyle_AZMX',
       'InteriorsStyle_DUFA', 'InteriorsStyle_7R8J', 'InteriorsStyle_H1CI',
       'InteriorsStyle_ARWN', 'ListDate_converted_2', 'ListDate_converted_3',
       'ListDate_converted_4', 'ListDate_converted_5', 'ListDate_converted_6',
       'ListDate_converted_7', 'ListDate_converted_8', 'ListDate_converted_9',
       'ListDate_co

### 19.Architecture

In [171]:
bd_all.Architecture.value_counts()

YIK5    17248
TP7R    12912
I3Z9    11305
AVYP     8844
1M5X     4258
4GZ4     2484
7IYX     2163
RZOR     2085
8SPA     1811
K92H     1644
KML6     1328
M3C0     1144
5NBT     1030
RIAX      979
STJH      913
X120      720
5A9R      649
DKOO      497
AZHP      243
FCXM      134
1IKD      134
WWBR      129
W86R       97
XDZM       42
R0QP       37
CR4E       33
BOLF       33
O8PU       31
GHDG       28
PQP2       24
OYQM        2
2MTV        1
OJBM        1
Name: Architecture, dtype: int64

In [172]:
# create dummies with cutoff of 3000

In [173]:
cols = ['Architecture']
for col in cols:
    k = bd_all[col].value_counts()
    k = k[k>3000]
    indexes = k.index[:-1]
    print(indexes)
    for i in indexes:
        print(i)
        bd_all[col+'_'+str(i)] = (bd_all[col]==i).astype(int)

Index(['YIK5', 'TP7R', 'I3Z9', 'AVYP'], dtype='object')
YIK5
TP7R
I3Z9
AVYP


In [174]:
bd_all.columns

Index(['Junk', 'InteriorsStyle', 'PriceIndex8', 'ListDate', 'Material',
       'PriceIndex9', 'Agency', 'AreaIncomeType', 'EnvRating', 'PriceIndex7',
       'ExpeditedListing', 'PriceIndex4', 'PriceIndex1', 'PriceIndex6',
       'PRIMEUNIT', 'Channel', 'Zip', 'InsurancePremiumIndex', 'PlotType',
       'Architecture', 'PriceIndex3', 'Region', 'PriceIndex5', 'SubModel',
       'Facade', 'State', 'NormalisedPopulation', 'BuildYear', 'RegionType',
       'PropertyAge', 'PriceIndex2', 'data', 'InteriorsStyle_K3BB',
       'InteriorsStyle_RCQE', 'InteriorsStyle_WEGU', 'InteriorsStyle_2XWO',
       'InteriorsStyle_V3Y1', 'InteriorsStyle_YBQJ', 'InteriorsStyle_AZMX',
       'InteriorsStyle_DUFA', 'InteriorsStyle_7R8J', 'InteriorsStyle_H1CI',
       'InteriorsStyle_ARWN', 'ListDate_converted_2', 'ListDate_converted_3',
       'ListDate_converted_4', 'ListDate_converted_5', 'ListDate_converted_6',
       'ListDate_converted_7', 'ListDate_converted_8', 'ListDate_converted_9',
       'ListDate_co

### 20.PriceIndex3

In [175]:
bd_all.PriceIndex3

0        13108.0
1         3695.0
2         7730.0
3         6290.0
4         8583.0
          ...   
10943     9677.0
10944     6860.0
10945    11515.0
10946    10475.0
10947     3251.0
Name: PriceIndex3, Length: 72983, dtype: object

In [176]:
#convert to numeric

In [177]:
bd_all.PriceIndex3=pd.to_numeric(bd_all.PriceIndex3,errors='coerce')

In [178]:
bd_all.PriceIndex3.dtype

dtype('float64')

### 21.Region

In [180]:
bd_all.Region.value_counts()

A          70398
B           2576
missing        9
Name: Region, dtype: int64

In [181]:
#create dummies with cutoff 2000 to elmininate B

In [182]:
cols = ['Region']
for col in cols:
    k = bd_all[col].value_counts()
    k = k[k>2000]
    indexes = k.index[:-1]
    print(indexes)
    for i in indexes:
        print(i)
        bd_all[col+'_'+str(i)] = (bd_all[col]==i).astype(int)

Index(['A'], dtype='object')
A


In [183]:
bd_all.columns

Index(['Junk', 'InteriorsStyle', 'PriceIndex8', 'ListDate', 'Material',
       'PriceIndex9', 'Agency', 'AreaIncomeType', 'EnvRating', 'PriceIndex7',
       'ExpeditedListing', 'PriceIndex4', 'PriceIndex1', 'PriceIndex6',
       'PRIMEUNIT', 'Channel', 'Zip', 'InsurancePremiumIndex', 'PlotType',
       'Architecture', 'PriceIndex3', 'Region', 'PriceIndex5', 'SubModel',
       'Facade', 'State', 'NormalisedPopulation', 'BuildYear', 'RegionType',
       'PropertyAge', 'PriceIndex2', 'data', 'InteriorsStyle_K3BB',
       'InteriorsStyle_RCQE', 'InteriorsStyle_WEGU', 'InteriorsStyle_2XWO',
       'InteriorsStyle_V3Y1', 'InteriorsStyle_YBQJ', 'InteriorsStyle_AZMX',
       'InteriorsStyle_DUFA', 'InteriorsStyle_7R8J', 'InteriorsStyle_H1CI',
       'InteriorsStyle_ARWN', 'ListDate_converted_2', 'ListDate_converted_3',
       'ListDate_converted_4', 'ListDate_converted_5', 'ListDate_converted_6',
       'ListDate_converted_7', 'ListDate_converted_8', 'ListDate_converted_9',
       'ListDate_co

### 22.PriceIndex5

In [184]:
bd_all.PriceIndex5

0         9022.0
1         2742.0
2         3768.0
3         5370.0
4         4803.0
          ...   
10943     8879.0
10944     3594.0
10945    10424.0
10946     6755.0
10947     2800.0
Name: PriceIndex5, Length: 72983, dtype: object

In [185]:
#convert to numeric

In [186]:
bd_all.PriceIndex5=pd.to_numeric(bd_all.PriceIndex5,errors='coerce')

In [187]:
bd_all.PriceIndex5.dtype

dtype('float64')

### 23.SubModel

In [189]:
bd_all.SubModel.value_counts()

FIVQ    15236
KT8F     4718
UA42     3859
8YTZ     2230
GR5P     1258
        ...  
QHCZ        1
SI6Y        1
AXKW        1
MQ43        1
ZKQ8        1
Name: SubModel, Length: 864, dtype: int64

In [190]:
#create dummies with cutoff of 3000

In [191]:
cols = ['SubModel']
for col in cols:
    k = bd_all[col].value_counts()
    k = k[k>3000]
    indexes = k.index[:-1]
    print(indexes)
    for i in indexes:
        print(i)
        bd_all[col+'_'+str(i)] = (bd_all[col]==i).astype(int)

Index(['FIVQ', 'KT8F'], dtype='object')
FIVQ
KT8F


In [193]:
bd_all.columns

Index(['Junk', 'InteriorsStyle', 'PriceIndex8', 'ListDate', 'Material',
       'PriceIndex9', 'Agency', 'AreaIncomeType', 'EnvRating', 'PriceIndex7',
       'ExpeditedListing', 'PriceIndex4', 'PriceIndex1', 'PriceIndex6',
       'PRIMEUNIT', 'Channel', 'Zip', 'InsurancePremiumIndex', 'PlotType',
       'Architecture', 'PriceIndex3', 'Region', 'PriceIndex5', 'SubModel',
       'Facade', 'State', 'NormalisedPopulation', 'BuildYear', 'RegionType',
       'PropertyAge', 'PriceIndex2', 'data', 'InteriorsStyle_K3BB',
       'InteriorsStyle_RCQE', 'InteriorsStyle_WEGU', 'InteriorsStyle_2XWO',
       'InteriorsStyle_V3Y1', 'InteriorsStyle_YBQJ', 'InteriorsStyle_AZMX',
       'InteriorsStyle_DUFA', 'InteriorsStyle_7R8J', 'InteriorsStyle_H1CI',
       'InteriorsStyle_ARWN', 'ListDate_converted_2', 'ListDate_converted_3',
       'ListDate_converted_4', 'ListDate_converted_5', 'ListDate_converted_6',
       'ListDate_converted_7', 'ListDate_converted_8', 'ListDate_converted_9',
       'ListDate_co

### 24.Facade

In [195]:
bd_all.Facade.value_counts()

SILVER       14875
WHITE        12123
BLUE         10347
GREY          7887
BLACK         7627
RED           6257
GOLD          5231
GREEN         3194
MAROON        2046
BEIGE         1584
BROWN          436
ORANGE         415
PURPLE         373
YELLOW         244
OTHER          242
NOT AVAIL       94
missing          8
Name: Facade, dtype: int64

In [196]:
#create dummies with frequency cutoff of 3000

In [197]:
cols = ['Facade']
for col in cols:
    k = bd_all[col].value_counts()
    k = k[k>3000]
    indexes = k.index[:-1]
    print(indexes)
    for i in indexes:
        print(i)
        bd_all[col+'_'+str(i)] = (bd_all[col]==i).astype(int)

Index(['SILVER', 'WHITE', 'BLUE', 'GREY', 'BLACK', 'RED', 'GOLD'], dtype='object')
SILVER
WHITE
BLUE
GREY
BLACK
RED
GOLD


In [198]:
bd_all.columns

Index(['Junk', 'InteriorsStyle', 'PriceIndex8', 'ListDate', 'Material',
       'PriceIndex9', 'Agency', 'AreaIncomeType', 'EnvRating', 'PriceIndex7',
       'ExpeditedListing', 'PriceIndex4', 'PriceIndex1', 'PriceIndex6',
       'PRIMEUNIT', 'Channel', 'Zip', 'InsurancePremiumIndex', 'PlotType',
       'Architecture', 'PriceIndex3', 'Region', 'PriceIndex5', 'SubModel',
       'Facade', 'State', 'NormalisedPopulation', 'BuildYear', 'RegionType',
       'PropertyAge', 'PriceIndex2', 'data', 'InteriorsStyle_K3BB',
       'InteriorsStyle_RCQE', 'InteriorsStyle_WEGU', 'InteriorsStyle_2XWO',
       'InteriorsStyle_V3Y1', 'InteriorsStyle_YBQJ', 'InteriorsStyle_AZMX',
       'InteriorsStyle_DUFA', 'InteriorsStyle_7R8J', 'InteriorsStyle_H1CI',
       'InteriorsStyle_ARWN', 'ListDate_converted_2', 'ListDate_converted_3',
       'ListDate_converted_4', 'ListDate_converted_5', 'ListDate_converted_6',
       'ListDate_converted_7', 'ListDate_converted_8', 'ListDate_converted_9',
       'ListDate_co

### 25.State

In [200]:
bd_all.State.value_counts()

TX    13596
FL    10447
CA     7095
NC     7042
AZ     6174
CO     4998
SC     4280
OK     3594
GA     2450
TN     1764
VA     1662
MD     1158
UT      875
PA      847
OH      795
MO      758
AL      690
NV      562
IA      499
MS      493
IN      486
IL      458
LA      349
NJ      317
WV      292
NM      239
KY      230
OR      211
ID      196
WA      136
NH       97
AR       70
MN       62
NE       26
MA       15
MI       14
NY        6
Name: State, dtype: int64

In [201]:
#create dummies with freq cutoff as 3000

In [202]:
cols = ['State']
for col in cols:
    k = bd_all[col].value_counts()
    k = k[k>3000]
    indexes = k.index[:-1]
    print(indexes)
    for i in indexes:
        print(i)
        bd_all[col+'_'+str(i)] = (bd_all[col]==i).astype(int)

Index(['TX', 'FL', 'CA', 'NC', 'AZ', 'CO', 'SC'], dtype='object')
TX
FL
CA
NC
AZ
CO
SC


In [203]:
bd_all.columns

Index(['Junk', 'InteriorsStyle', 'PriceIndex8', 'ListDate', 'Material',
       'PriceIndex9', 'Agency', 'AreaIncomeType', 'EnvRating', 'PriceIndex7',
       'ExpeditedListing', 'PriceIndex4', 'PriceIndex1', 'PriceIndex6',
       'PRIMEUNIT', 'Channel', 'Zip', 'InsurancePremiumIndex', 'PlotType',
       'Architecture', 'PriceIndex3', 'Region', 'PriceIndex5', 'SubModel',
       'Facade', 'State', 'NormalisedPopulation', 'BuildYear', 'RegionType',
       'PropertyAge', 'PriceIndex2', 'data', 'InteriorsStyle_K3BB',
       'InteriorsStyle_RCQE', 'InteriorsStyle_WEGU', 'InteriorsStyle_2XWO',
       'InteriorsStyle_V3Y1', 'InteriorsStyle_YBQJ', 'InteriorsStyle_AZMX',
       'InteriorsStyle_DUFA', 'InteriorsStyle_7R8J', 'InteriorsStyle_H1CI',
       'InteriorsStyle_ARWN', 'ListDate_converted_2', 'ListDate_converted_3',
       'ListDate_converted_4', 'ListDate_converted_5', 'ListDate_converted_6',
       'ListDate_converted_7', 'ListDate_converted_8', 'ListDate_converted_9',
       'ListDate_co

### 26.NormalisedPopulation

In [204]:
bd_all.NormalisedPopulation

0        42077
1        77258
2        94514
3        82302
4        49176
         ...  
10943    51629
10944    69990
10945    73737
10946    85960
10947    69307
Name: NormalisedPopulation, Length: 72983, dtype: int64

In [205]:
#no changes required

### 27.BuildYear

In [207]:
bd_all.BuildYear.value_counts()

2006    17043
2005    15489
2007    11423
2004    10207
2008     6885
2003     6227
2002     3405
2001     1481
2009      822
2010        1
Name: BuildYear, dtype: int64

In [208]:
#create dummies with freq cutoff has 3000

In [209]:
cols = ['BuildYear']
for col in cols:
    k = bd_all[col].value_counts()
    k = k[k>3000]
    indexes = k.index[:-1]
    print(indexes)
    for i in indexes:
        print(i)
        bd_all[col+'_'+str(i)] = (bd_all[col]==i).astype(int)

Int64Index([2006, 2005, 2007, 2004, 2008, 2003], dtype='int64')
2006
2005
2007
2004
2008
2003


In [210]:
bd_all.columns

Index(['Junk', 'InteriorsStyle', 'PriceIndex8', 'ListDate', 'Material',
       'PriceIndex9', 'Agency', 'AreaIncomeType', 'EnvRating', 'PriceIndex7',
       'ExpeditedListing', 'PriceIndex4', 'PriceIndex1', 'PriceIndex6',
       'PRIMEUNIT', 'Channel', 'Zip', 'InsurancePremiumIndex', 'PlotType',
       'Architecture', 'PriceIndex3', 'Region', 'PriceIndex5', 'SubModel',
       'Facade', 'State', 'NormalisedPopulation', 'BuildYear', 'RegionType',
       'PropertyAge', 'PriceIndex2', 'data', 'InteriorsStyle_K3BB',
       'InteriorsStyle_RCQE', 'InteriorsStyle_WEGU', 'InteriorsStyle_2XWO',
       'InteriorsStyle_V3Y1', 'InteriorsStyle_YBQJ', 'InteriorsStyle_AZMX',
       'InteriorsStyle_DUFA', 'InteriorsStyle_7R8J', 'InteriorsStyle_H1CI',
       'InteriorsStyle_ARWN', 'ListDate_converted_2', 'ListDate_converted_3',
       'ListDate_converted_4', 'ListDate_converted_5', 'ListDate_converted_6',
       'ListDate_converted_7', 'ListDate_converted_8', 'ListDate_converted_9',
       'ListDate_co

### 28.RegionType

In [212]:
bd_all.RegionType.value_counts()

A          61028
B           8033
C           3722
OTHER        195
missing        5
Name: RegionType, dtype: int64

In [213]:
#create dummies with cutoff of 3000

In [214]:
cols = ['RegionType']
for col in cols:
    k = bd_all[col].value_counts()
    k = k[k>3000]
    indexes = k.index[:-1]
    print(indexes)
    for i in indexes:
        print(i)
        bd_all[col+'_'+str(i)] = (bd_all[col]==i).astype(int)

Index(['A', 'B'], dtype='object')
A
B


In [215]:
bd_all.columns

Index(['Junk', 'InteriorsStyle', 'PriceIndex8', 'ListDate', 'Material',
       'PriceIndex9', 'Agency', 'AreaIncomeType', 'EnvRating', 'PriceIndex7',
       ...
       'State_CO', 'State_SC', 'BuildYear_2006', 'BuildYear_2005',
       'BuildYear_2007', 'BuildYear_2004', 'BuildYear_2008', 'BuildYear_2003',
       'RegionType_A', 'RegionType_B'],
      dtype='object', length=102)

### 29.PropertyAge

In [216]:
bd_all.PropertyAge

0        2
1        7
2        6
3        7
4        3
        ..
10943    1
10944    4
10945    2
10946    3
10947    5
Name: PropertyAge, Length: 72983, dtype: int64

In [217]:
#no changes required

### 30.PriceIndex2

In [218]:
bd_all.PriceIndex2

0        10692.0
1         3615.0
2         5805.0
3         6444.0
4         6286.0
          ...   
10943     9181.0
10944     5185.0
10945    12088.0
10946     7978.0
10947     3447.0
Name: PriceIndex2, Length: 72983, dtype: object

In [219]:
#convert to numeric

In [220]:
bd_all.PriceIndex2=pd.to_numeric(bd_all.PriceIndex2,errors='coerce')

In [221]:
bd_all.PriceIndex2.dtype

dtype('float64')

In [222]:
bd_all.shape

(72983, 102)

# Copy the data

In [237]:
bd_all.columns


Index(['Junk', 'InteriorsStyle', 'PriceIndex8', 'ListDate', 'Material',
       'PriceIndex9', 'Agency', 'AreaIncomeType', 'EnvRating', 'PriceIndex7',
       ...
       'State_CO', 'State_SC', 'BuildYear_2006', 'BuildYear_2005',
       'BuildYear_2007', 'BuildYear_2004', 'BuildYear_2008', 'BuildYear_2003',
       'RegionType_A', 'RegionType_B'],
      dtype='object', length=102)

In [240]:
bd_selected_cols=['Junk','PriceIndex8','PriceIndex9','PriceIndex7',
       'ExpeditedListing', 'PriceIndex4', 'PriceIndex1', 'PriceIndex6',
       'InsurancePremiumIndex','PriceIndex3','PriceIndex5',
       'NormalisedPopulation',
       'PropertyAge', 'PriceIndex2', 'data', 'InteriorsStyle_K3BB',
       'InteriorsStyle_RCQE', 'InteriorsStyle_WEGU', 'InteriorsStyle_2XWO',
       'InteriorsStyle_V3Y1', 'InteriorsStyle_YBQJ', 'InteriorsStyle_AZMX',
       'InteriorsStyle_DUFA', 'InteriorsStyle_7R8J', 'InteriorsStyle_H1CI',
       'InteriorsStyle_ARWN', 'ListDate_converted_2', 'ListDate_converted_3',
       'ListDate_converted_4', 'ListDate_converted_5', 'ListDate_converted_6',
       'ListDate_converted_7', 'ListDate_converted_8', 'ListDate_converted_9',
       'ListDate_converted_10', 'ListDate_converted_11',
       'ListDate_converted_12', 'Material_PRN0', 'Material_C8A4',
       'Material_ZCI3', 'Material_NYFD', 'Agency_CAT1', 'Agency_CAT2',
       'Agency_CAT3', 'AreaIncomeType_A', 'AreaIncomeType_B',
       'EnvRating_missing', 'PRIMEUNIT_missing', 'Channel_Direct',
       'Channel_Other', 'Zip_32824', 'PlotType_WHBI', 'PlotType_W62B',
       'PlotType_32T2', 'PlotType_DIJX', 'PlotType_WLQ6', 'Architecture_YIK5',
       'Architecture_TP7R', 'Architecture_I3Z9', 'Architecture_AVYP',
       'Region_A', 'SubModel_FIVQ', 'SubModel_KT8F', 'Facade_SILVER',
       'Facade_WHITE', 'Facade_BLUE', 'Facade_GREY', 'Facade_BLACK',
       'Facade_RED', 'Facade_GOLD', 'State_TX', 'State_FL', 'State_CA',
       'State_NC', 'State_AZ', 'State_CO', 'State_SC', 'BuildYear_2006',
       'BuildYear_2005', 'BuildYear_2007', 'BuildYear_2004', 'BuildYear_2008',
       'BuildYear_2003','RegionType_A', 'RegionType_B']

bd_selected_cols

['Junk',
 'PriceIndex8',
 'PriceIndex9',
 'PriceIndex7',
 'ExpeditedListing',
 'PriceIndex4',
 'PriceIndex1',
 'PriceIndex6',
 'InsurancePremiumIndex',
 'PriceIndex3',
 'PriceIndex5',
 'NormalisedPopulation',
 'PropertyAge',
 'PriceIndex2',
 'data',
 'InteriorsStyle_K3BB',
 'InteriorsStyle_RCQE',
 'InteriorsStyle_WEGU',
 'InteriorsStyle_2XWO',
 'InteriorsStyle_V3Y1',
 'InteriorsStyle_YBQJ',
 'InteriorsStyle_AZMX',
 'InteriorsStyle_DUFA',
 'InteriorsStyle_7R8J',
 'InteriorsStyle_H1CI',
 'InteriorsStyle_ARWN',
 'ListDate_converted_2',
 'ListDate_converted_3',
 'ListDate_converted_4',
 'ListDate_converted_5',
 'ListDate_converted_6',
 'ListDate_converted_7',
 'ListDate_converted_8',
 'ListDate_converted_9',
 'ListDate_converted_10',
 'ListDate_converted_11',
 'ListDate_converted_12',
 'Material_PRN0',
 'Material_C8A4',
 'Material_ZCI3',
 'Material_NYFD',
 'Agency_CAT1',
 'Agency_CAT2',
 'Agency_CAT3',
 'AreaIncomeType_A',
 'AreaIncomeType_B',
 'EnvRating_missing',
 'PRIMEUNIT_missing',
 '

In [239]:
bd_all_copy=bd_all.copy()

In [241]:
bd_all_copy=bd_all_copy[bd_selected_cols]

In [242]:
bd_all_copy.head()

,Junk,PriceIndex8,PriceIndex9,PriceIndex7,ExpeditedListing,PriceIndex4,PriceIndex1,PriceIndex6,InsurancePremiumIndex,PriceIndex3,...,State_CO,State_SC,BuildYear_2006,BuildYear_2005,BuildYear_2007,BuildYear_2004,BuildYear_2008,BuildYear_2003,RegionType_A,RegionType_B
0,0.0,14674.0,8270,13143.0,0,14224.0,9217.0,10387.0,623,13108.0,...,0,0,0,0,0,0,1,0,1,0
1,1.0,4172.0,3890,3461.0,0,4404.0,2958.0,3400.0,1689,3695.0,...,0,0,0,0,0,0,0,0,1,0
2,0.0,8210.0,5900,6922.0,0,8952.0,4568.0,4862.0,2351,7730.0,...,0,1,0,0,0,1,0,0,1,0
3,0.0,7309.0,7460,6300.0,0,7460.0,5361.0,6305.0,1933,6290.0,...,0,0,0,0,0,0,0,0,0,0
4,0.0,9182.0,6400,8330.0,0,9846.0,5428.0,5718.0,482,8583.0,...,0,1,0,0,1,0,0,0,0,1


In [245]:
bd_all_copy.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 72983 entries, 0 to 10947
Data columns (total 85 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Junk                   62035 non-null  float64
 1   PriceIndex8            72668 non-null  float64
 2   PriceIndex9            72983 non-null  int64  
 3   PriceIndex7            72668 non-null  float64
 4   ExpeditedListing       72983 non-null  int64  
 5   PriceIndex4            72965 non-null  float64
 6   PriceIndex1            72965 non-null  float64
 7   PriceIndex6            72668 non-null  float64
 8   InsurancePremiumIndex  72983 non-null  int64  
 9   PriceIndex3            72965 non-null  float64
 10  PriceIndex5            72668 non-null  float64
 11  NormalisedPopulation   72983 non-null  int64  
 12  PropertyAge            72983 non-null  int64  
 13  PriceIndex2            72965 non-null  float64
 14  data                   72983 non-null  object 
 15  In

## Working on null values

In [246]:
list(zip(bd_all_copy.columns,bd_all_copy.dtypes,bd_all_copy.isnull().sum()))

[('Junk', dtype('float64'), 10948),
 ('PriceIndex8', dtype('float64'), 315),
 ('PriceIndex9', dtype('int64'), 0),
 ('PriceIndex7', dtype('float64'), 315),
 ('ExpeditedListing', dtype('int64'), 0),
 ('PriceIndex4', dtype('float64'), 18),
 ('PriceIndex1', dtype('float64'), 18),
 ('PriceIndex6', dtype('float64'), 315),
 ('InsurancePremiumIndex', dtype('int64'), 0),
 ('PriceIndex3', dtype('float64'), 18),
 ('PriceIndex5', dtype('float64'), 315),
 ('NormalisedPopulation', dtype('int64'), 0),
 ('PropertyAge', dtype('int64'), 0),
 ('PriceIndex2', dtype('float64'), 18),
 ('data', dtype('O'), 0),
 ('InteriorsStyle_K3BB', dtype('int32'), 0),
 ('InteriorsStyle_RCQE', dtype('int32'), 0),
 ('InteriorsStyle_WEGU', dtype('int32'), 0),
 ('InteriorsStyle_2XWO', dtype('int32'), 0),
 ('InteriorsStyle_V3Y1', dtype('int32'), 0),
 ('InteriorsStyle_YBQJ', dtype('int32'), 0),
 ('InteriorsStyle_AZMX', dtype('int32'), 0),
 ('InteriorsStyle_DUFA', dtype('int32'), 0),
 ('InteriorsStyle_7R8J', dtype('int32'), 0),


In [247]:
# priceInde series only has null values so replace it with median

In [248]:
cat=['PriceIndex'+str(i) for i in range(1,10)]
cat

['PriceIndex1',
 'PriceIndex2',
 'PriceIndex3',
 'PriceIndex4',
 'PriceIndex5',
 'PriceIndex6',
 'PriceIndex7',
 'PriceIndex8',
 'PriceIndex9']

In [250]:
for col in cat:
        bd_all_copy[col]=bd_all_copy[col].fillna(bd_all_copy.loc[bd_all_copy['data']=='train',col].median())

In [251]:
list(zip(bd_all_copy.columns,bd_all_copy.dtypes,bd_all_copy.isnull().sum()))

[('Junk', dtype('float64'), 10948),
 ('PriceIndex8', dtype('float64'), 0),
 ('PriceIndex9', dtype('int64'), 0),
 ('PriceIndex7', dtype('float64'), 0),
 ('ExpeditedListing', dtype('int64'), 0),
 ('PriceIndex4', dtype('float64'), 0),
 ('PriceIndex1', dtype('float64'), 0),
 ('PriceIndex6', dtype('float64'), 0),
 ('InsurancePremiumIndex', dtype('int64'), 0),
 ('PriceIndex3', dtype('float64'), 0),
 ('PriceIndex5', dtype('float64'), 0),
 ('NormalisedPopulation', dtype('int64'), 0),
 ('PropertyAge', dtype('int64'), 0),
 ('PriceIndex2', dtype('float64'), 0),
 ('data', dtype('O'), 0),
 ('InteriorsStyle_K3BB', dtype('int32'), 0),
 ('InteriorsStyle_RCQE', dtype('int32'), 0),
 ('InteriorsStyle_WEGU', dtype('int32'), 0),
 ('InteriorsStyle_2XWO', dtype('int32'), 0),
 ('InteriorsStyle_V3Y1', dtype('int32'), 0),
 ('InteriorsStyle_YBQJ', dtype('int32'), 0),
 ('InteriorsStyle_AZMX', dtype('int32'), 0),
 ('InteriorsStyle_DUFA', dtype('int32'), 0),
 ('InteriorsStyle_7R8J', dtype('int32'), 0),
 ('Interiors

#### data preparation is over

# ML part

In [252]:
target='Junk'

In [253]:
x_train=bd_all_copy.drop([target,'data'],1)[bd_all_copy['data']=='train']
y_train=bd_all_copy[target][bd_all_copy['data']=='train']
x_test=bd_all_copy.drop([target,'data'],1)[bd_all_copy['data']=='test']

In [254]:
x_train.head()

,PriceIndex8,PriceIndex9,PriceIndex7,ExpeditedListing,PriceIndex4,PriceIndex1,PriceIndex6,InsurancePremiumIndex,PriceIndex3,PriceIndex5,...,State_CO,State_SC,BuildYear_2006,BuildYear_2005,BuildYear_2007,BuildYear_2004,BuildYear_2008,BuildYear_2003,RegionType_A,RegionType_B
0,14674.0,8270,13143.0,0,14224.0,9217.0,10387.0,623,13108.0,9022.0,...,0,0,0,0,0,0,1,0,1,0
1,4172.0,3890,3461.0,0,4404.0,2958.0,3400.0,1689,3695.0,2742.0,...,0,0,0,0,0,0,0,0,1,0
2,8210.0,5900,6922.0,0,8952.0,4568.0,4862.0,2351,7730.0,3768.0,...,0,1,0,0,0,1,0,0,1,0
3,7309.0,7460,6300.0,0,7460.0,5361.0,6305.0,1933,6290.0,5370.0,...,0,0,0,0,0,0,0,0,0,0
4,9182.0,6400,8330.0,0,9846.0,5428.0,5718.0,482,8583.0,4803.0,...,0,1,0,0,1,0,0,0,0,1


In [256]:
y_train.head()

0    0.0
1    1.0
2    0.0
3    0.0
4    0.0
Name: Junk, dtype: float64

In [257]:
x_test.head()

,PriceIndex8,PriceIndex9,PriceIndex7,ExpeditedListing,PriceIndex4,PriceIndex1,PriceIndex6,InsurancePremiumIndex,PriceIndex3,PriceIndex5,...,State_CO,State_SC,BuildYear_2006,BuildYear_2005,BuildYear_2007,BuildYear_2004,BuildYear_2008,BuildYear_2003,RegionType_A,RegionType_B
0,10645.0,6015,9568.0,0,12432.0,6986.0,7133.0,728,10860.0,5911.0,...,0,0,1,0,0,0,0,0,1,0
1,9891.0,7625,8888.0,0,12173.0,5954.0,6544.0,1334,10166.0,5370.0,...,0,0,0,0,0,1,0,0,0,1
2,8426.0,6900,7911.0,0,9134.0,7407.0,7339.0,754,8500.0,6862.0,...,0,0,0,0,0,0,1,0,1,0
3,9269.0,6475,7773.0,0,8500.0,4241.0,6143.0,1373,7294.0,4872.0,...,1,0,0,1,0,0,0,0,1,0
4,5366.0,3960,4134.0,0,5211.0,3352.0,4506.0,2939,4120.0,3365.0,...,0,0,0,0,0,0,0,1,1,0


### XGB

In [260]:
param_dist = {
              "max_depth": [2,3,4,5,6],
              "learning_rate":[0.01,0.05,0.1,0.3,0.5],
                "min_child_weight":[4,5,6],
              "subsample":[i/10.0 for i in range(6,10)],
                "colsample_bytree":[i/10.0 for i in range(6,10)],
               "reg_alpha":[1e-5, 1e-2, 0.1, 1, 100],
              "gamma":[i/10.0 for i in range(0,5)],
            "n_estimators":[100,500,700,1000],
            'scale_pos_weight':[2,3,4,5,6,7,8,9]
              }

In [261]:
clf=XGBClassifier(objective='binary:logistic')

In [263]:
n_iter=25

random_search=RandomizedSearchCV(clf,n_jobs=-1,verbose=3,cv=10,n_iter=n_iter,scoring='roc_auc',
                                 param_distributions=param_dist)

In [264]:
random_search.fit(x_train,y_train)

Fitting 10 folds for each of 25 candidates, totalling 250 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed: 24.7min
[Parallel(n_jobs=-1)]: Done 120 tasks      | elapsed: 132.9min
[Parallel(n_jobs=-1)]: Done 250 out of 250 | elapsed: 237.1min finished


RandomizedSearchCV(cv=10, error_score=nan,
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None, gamma=None,
                                           gpu_id=None, importance_type='gain',
                                           interaction_constraints=None,
                                           learning_rate=None,
                                           max_delta_step=None, max_depth=None,
                                           min_child_weight=None, missing=nan,
                                           monotone_constraints=None,...
                                        'gamma': [0.0, 0.1, 0.2, 0.3, 0.4],
                                        'learning_rate': [0.01, 0.05, 0.1, 0.3,
                                                          0.5],
    

In [265]:
def report(results,n_top=3):
    for i in range(1,n_top+1):
        candidates = np.flatnonzero(results['rank_test_score']==i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean Validation Score: {0:.8f} (std:{1:.3f})".format(
                results['mean_test_score'][candidate],
                results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")

In [266]:
report(random_search.cv_results_,5)

Model with rank: 1
Mean Validation Score: 0.76959115 (std:0.007)
Parameters: {'subsample': 0.7, 'scale_pos_weight': 6, 'reg_alpha': 0.01, 'n_estimators': 700, 'min_child_weight': 6, 'max_depth': 3, 'learning_rate': 0.05, 'gamma': 0.2, 'colsample_bytree': 0.8}

Model with rank: 2
Mean Validation Score: 0.76886093 (std:0.007)
Parameters: {'subsample': 0.6, 'scale_pos_weight': 8, 'reg_alpha': 0.1, 'n_estimators': 700, 'min_child_weight': 6, 'max_depth': 3, 'learning_rate': 0.05, 'gamma': 0.2, 'colsample_bytree': 0.6}

Model with rank: 3
Mean Validation Score: 0.76841805 (std:0.007)
Parameters: {'subsample': 0.8, 'scale_pos_weight': 6, 'reg_alpha': 0.01, 'n_estimators': 1000, 'min_child_weight': 6, 'max_depth': 4, 'learning_rate': 0.01, 'gamma': 0.3, 'colsample_bytree': 0.7}

Model with rank: 4
Mean Validation Score: 0.76785410 (std:0.008)
Parameters: {'subsample': 0.8, 'scale_pos_weight': 3, 'reg_alpha': 100, 'n_estimators': 700, 'min_child_weight': 6, 'max_depth': 5, 'learning_rate': 0.0

In [ ]:
#here we choose 1 st rank

Model with rank: 1
Mean Validation Score: 0.76959115 (std:0.007)
Parameters: {'subsample': 0.7, 'scale_pos_weight': 6, 'reg_alpha': 0.01,
             'n_estimators': 700, 'min_child_weight': 6, 'max_depth': 3, 'learning_rate': 0.05,
             'gamma': 0.2, 'colsample_bytree': 0.8}

In [267]:
xgb_best=XGBClassifier(subsample=0.7,scale_pos_weight=6,reg_alpha=0.01,n_estimators=700,min_child_weight=6,
                       max_depth=3,learning_rate=0.05,gamma=0.2,colsample_bytree=0.8,n_jobs=-1)

In [268]:
xgb_best.fit(x_train,y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.8, gamma=0.2, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.05, max_delta_step=0, max_depth=3,
              min_child_weight=6, missing=nan, monotone_constraints='()',
              n_estimators=700, n_jobs=-1, num_parallel_tree=1,
              objective='binary:logistic', random_state=0, reg_alpha=0.01,
              reg_lambda=1, scale_pos_weight=6, subsample=0.7,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [269]:
submissions=pd.DataFrame({'Junk':xgb_best.predict_proba(x_test)[:,1]})

In [270]:
submissions.to_csv('Surendran_R_attempt_1_project_5.csv',index=False)

In [271]:
r=pd.read_csv('Surendran_R_attempt_1_project_5.csv')

In [272]:
r.shape

(10948, 1)

In [273]:
x_test.shape

(10948, 83)

In [274]:
r.head()

,Junk
0,0.274136
1,0.462690
2,0.113499
3,0.286375
4,0.666402
